<a href="https://colab.research.google.com/github/Omar-Basel-ALkhasawneh/JoVision_AI-Tasks/blob/main/Task6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
!pip install rarfile
!apt-get install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
import rarfile
import os


rarPath1 = '/content/TIRES.rar'
rarPath2 = '/content/random.rar'

extractPath1 = '/content/'
extractPath2 = '/content/Random'

with rarfile.RarFile(rarPath1) as rf:
    rf.extractall(extractPath1)

with rarfile.RarFile(rarPath2) as rf:
    rf.extractall(extractPath2)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import RandomFlip, RandomRotation, GaussianNoise



trainingPath = '/content/TIRES'
testingPath = '/content/Random'


train_dataset = tf.keras.utils.image_dataset_from_directory(
    trainingPath,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(300, 300),
    batch_size=32
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    trainingPath,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(300, 300),
    batch_size=32
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    testingPath,
    image_size=(300, 300),
    batch_size=32,
    shuffle=False
)


normalization_layer = tf.keras.layers.Rescaling(1./255)

data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal_and_vertical"),
    RandomRotation(0.2),
    GaussianNoise(0.1)
])

class_names = train_dataset.class_names

train_dataset = train_dataset.map(lambda x, y: (data_augmentation(normalization_layer(x)), tf.keras.utils.to_categorical(y, num_classes=4)))
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), tf.keras.utils.to_categorical(y, num_classes=4)))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), tf.keras.utils.to_categorical(y, num_classes=4)))

train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)



Found 3000 files belonging to 4 classes.
Using 2400 files for training.
Found 3000 files belonging to 4 classes.
Using 600 files for validation.
Found 2699 files belonging to 1 classes.


In [ ]:
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=50,
)

Epoch 1/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 268s 1s/step - accuracy: 0.9094 - loss: 0.3355 - val_accuracy: 0.4050 - val_loss: 2.1711
Epoch 2/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.9700 - loss: 0.1192 - val_accuracy: 0.4050 - val_loss: 4.9744
Epoch 3/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.9908 - loss: 0.0399 - val_accuracy: 0.3900 - val_loss: 3.5524
Epoch 4/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.9841 - loss: 0.0688 - val_accuracy: 0.4467 - val_loss: 2.2578
Epoch 5/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.9896 - loss: 0.0259 - val_accuracy: 0.4550 - val_loss: 2.0192
Epoch 6/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.9857 - loss: 0.0397 - val_accuracy: 0.9683 - val_loss: 0.1488
Epoch 7/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.9968 - loss: 0.0218 - val_accuracy: 0.9900 - val_loss: 0.0516
Epoch 8/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.9958 - loss: 0.0175 - val_accuracy: 0.9867 - val_loss

In [ ]:
predictions = model.predict(test_dataset)

predicted_class_indices = predictions.argmax(axis=-1)

filenames = sorted(os.listdir(r'/content/Random/random'))

labelsDict = {filenames[i]: labels[predicted_class_indices[i]] for i in range(len(filenames))}

for filename, prediction in labelsDict.items():
    print(f"{filename}: {prediction}")


85/85 ━━━━━━━━━━━━━━━━━━━━ 13s 151ms/step
14038843825.JPG: MultiTires
140422934160.JPG: MultiTires
140423293635.JPG: Side
140424023629.JPG: MultiTires
140424790626.JPG: MultiTires
140441004586.JPG: MultiTires
140483604855.JPG: MultiTires
140484913180.JPG: MultiTires
140501173862.JPG: MultiTires
140510444142.JPG: Side
140510482877.JPG: Side
140510673837.JPG: MultiTires
140544067164.JPG: MultiTires
140545042860.JPG: MultiTires
140596269813.JPG: MultiTires
140605904483.JPG: MultiTires
140630795691.JPG: Side
140631140781.JPG: MultiTires
140665219438.JPG: Side
140665689016.JPG: Side
14066576948.JPG: MultiTires
140691156330.JPG: MultiTires
14069120982.JPG: MultiTires
140699513911.JPG: MultiTires
140717444818.JPG: MultiTires
140717840475.JPG: MultiTires
140717941253.JPG: MultiTires
140752863925.JPG: MultiTires
140760770565.JPG: MultiTires
140786773494.JPG: OCR
14078779228.JPG: MultiTires
140795344436.JPG: MultiTires
140795548810.JPG: MultiTires
140795638872.JPG: MultiTires
140796298532.JPG: M